# Understand data warehouse fundamentals
The process of building a modern data warehouse typically consists of:
- **Data ingestion** - **`moving`** data from source systems into a data warehouse.
- **Data storage** - **`storing`** the data in a format that is optimized for analytics.
- **Data processing** - **`transforming`** the data into a format that is ready for consumption by analytical tools.
- **Data analysis and delivery** - **`analyzing`** the data to gain insights and delivering those insights to the business.

## Understand Fabric's data warehouse experience
**Fabric's data warehouse:** 
- Is a **`relational data warehouse`** that supports the full transactional T-SQL capabilities you'd expect from an enterprise data warehouse. 
- It's a fully managed, scalable, and highly available data warehouse that can be used to store and query data in the **`Lakehouse`**. 
  - Using the data warehouse, you're fully in control of creating tables, loading, transforming, and querying data using either the Fabric portal or T-SQL commands. 
  - You can use SQL to query and analyze the data, or use Spark to process the data and create machine learning models.

Data warehouses in Fabric facilitate collaboration between data engineers and data analysts, working together in the same experience. 
- Data engineers build a relational layer on top of data in the Lakehouse, where analysts can use T-SQL and Power BI to explore the data.

## Design a data warehouse

### Tables in a data warehouse
Tables in a data warehouse are typically organized in **`dimensional modeling`**, which involves structuring tables into **`fact tables`** and **`dimension tables`**.

**1. Fact tables:** contain the **`numerical data`** that you want to analyze. 
- Fact tables typically have a **large number** of rows and are the **primary source** of data for analysis. 
- For example, a fact table might contain the total amount paid for sales orders that occurred on a specific date or at a particular store.

**2. Dimension tables:** contain **`descriptive information`** about the data in the fact tables.
- Dimension tables typically have a **small number of rows** and are used to **provide context** for the data in the fact tables. 
- For example, a dimension table might contain information about the customers who placed sales orders.

It's common for a dimension table to include two key columns:

**1. A surrogate key:** is a unique identifier for each row in the dimension table. 
- It's often an integer value that is **`automatically generated`** by the database management system when a new row is inserted into the table.
- Surrogate keys are specific to the data warehouse and help to maintain consistency and accuracy in the data. 

**2. An alternate key:** is often a **`natural or business key`** that identifies a specific instance of an entity in the transactional source system - such as a product code or a customer ID.
- Alternate keys on the other hand are specific to the source system and help to maintain traceability between the data warehouse and the source system.

### Special types of dimension tables
Special types of dimensions provide additional context and enable more comprehensive data analysis.

**Time dimensions:** provide information about the **`time period`** in which an event occurred. 
- This table enables data analysts to aggregate data over temporal intervals. 
- For example, a time dimension might include columns for the year, quarter, month, and day in which a sales order was placed.

**Slowly changing dimensions:** are dimension tables that **`track changes`** to dimension attributes over time, like changes to a customer's address or a product's price. 
- They're significant in a data warehouse because they enable users to analyze and understand **changes to data over time**. 
- Slowly changing dimensions ensure that data stays up-to-date and accurate, which is imperative to making good business decisions.

## Data warehouse schema designs
In a data warehouse however, the dimension data is generally **`de-normalized`** to **reduce the number of joins** required to query the data.

**1. Star Schema:** Often, a data warehouse is organized as a **`star schema`**, in which a fact table is directly related to the dimension tables, as shown in this example:

![Star Schema](./images/10/star-schema.png)

You can use the attributes of something to group together numbers in the fact table at different levels. For example, you could find the total sales revenue for a whole region or just for one customer. The information for each level can be stored in the same dimension table.

<img src="https://files.training.databricks.com/images/icon_note_32.png" alt="Note"> See [What is a star schema?](https://learn.microsoft.com/en-us/power-bi/guidance/star-schema) for more information on designing star schemas for Fabric.

**2. Snowflake:**
If there are lots of levels or some information is shared by different things, it might make sense to use a **`snowflake`** schema instead. Here's an example:

![Snowflake](./images/10/snowflake-schema.png)

In this case, 
- The **DimProduct** table has been **`split up (normalized)`** to create separate dimension tables for product categories and suppliers.
  - Each row in the **DimProduct** table contains key values for the corresponding rows in the **DimCategory** and **DimSupplier** tables.
- A **DimGeography** table has been **`added`** containing information on where customers and stores are located.
  - Each row in the **DimCustomer** and **DimStore** tables contains a key value for the corresponding row in the **DimGeography** table.

# Understand data warehouses in Fabric
- **Fabric's Lakehouse:** is a collection of files, folders, tables, and shortcuts that act like a database over a **`data lake`**.
- **Fabric's data warehouse:** experience allows you to transition from the lake view of the Lakehouse (which supports data engineering and Apache Spark) to the **SQL experiences** that a traditional data warehouse would provide. 
- The **Lakehouse** gives you the ability to **`read`** tables and use the SQL analytics endpoint, whereas the **data warehouse** enables you to **`manipulate`** the data.
- In the data warehouse experience, you'll:
  - **model** data using tables and views
  - run T-SQL to **query data** across the data warehouse and Lakehouse
  - use T-SQL to **perform DML operations** on data inside the data warehouse
  - **serve** reporting layers like Power BI

## Describe a data warehouse in Fabric
In the data warehouse experience in Fabric, you can build a **relational layer** on top of **physical data** in the Lakehouse and expose it to analysis and reporting tools. 
- You can create your data warehouse directly in Fabric from the **`create hub`** or within a **`workspace`**. After creating an empty warehouse, you can add objects to it.
- Once your warehouse is created, you can create tables using T-SQL directly in the Fabric interface.

![Create Date Warehouse](./images/10/create-data-warehouse.png)

## Ingest data into your data warehouse
There are a few ways to ingest data into a Fabric data warehouse, including **Pipelines**, **Dataflows**, **cross-database querying**, and the **`COPY INTO`** command**.

### Create tables
To create a table in the data warehouse, you can use **SQL Server Management Studio (SSMS)** or another **SQL client** to connect to the data warehouse and run a **`CREATE TABLE`** statement. You can also create tables directly in the Fabric UI.

You can copy data from an external location into a table in the data warehouse using the **`COPY INTO`** syntax. For example:

In [0]:
%sql
COPY INTO dbo.Region 
FROM 'https://mystorageaccountxxx.blob.core.windows.net/private/Region.csv' WITH ( 
            FILE_TYPE = 'CSV'
            ,CREDENTIAL = ( 
                IDENTITY = 'Shared Access Signature'
                , SECRET = 'xxx'
                )
            ,FIRSTROW = 2
            )
GO

This SQL query loads data from a CSV file stored in Azure Blob Storage into a table called "Region" in the Fabric data warehouse.

![Create Table](./images/10/create-table-manual.png)

### Table considerations
**1. Staging tables:** are temporary tables that can be used to perform **data `cleansing`**, **data `transformations`**, and **data `validation`**.
- After creating tables in a data warehouse, it's important to consider the process of loading data into those tables. A common approach is to use **`staging tables`**
  - In Fabric, you can use T-SQL commands to load data from files into staging tables in the data warehouse.
  - You can also use staging tables to load data from multiple sources into a **single destination table**.

**2. Time interval:** Usually, data loading is performed as a **`periodic batch process`** in which inserts and updates to the data warehouse are coordinated to occur at a **`regular interval`** (for example, daily, weekly, or monthly).

**3. Steps:** Generally, you should implement a data warehouse load process that performs tasks in the following order:
1. Ingest the new data to be loaded into a **data lake**, applying **pre-load `cleansing`** or **`transformations`** as required.
2. Load the data from files into **`staging tables`** in the relational data warehouse.
3. Load the **`dimension tables`** from the dimension data in the **staging tables**, **updating** existing rows or **inserting** new rows and **generating `surrogate`** key values as necessary.
4. Load the **`fact tables`** from the fact data in the **staging tables**, looking up the appropriate **`surrogate`** keys for related dimensions.
5. Perform post-load optimization by **updating** **`indexes`** and table **`distribution statistics`**.

If you have tables in the lakehouse, and you want to be able to query it in your warehouse - but not make changes - with a Fabric data warehouse, you don't have to copy data from the lakehouse to the data warehouse. You can query data in the lakehouse directly from the data warehouse using cross-database querying.

<img src="https://files.training.databricks.com/images/icon_note_32.png" alt="Note"> Working with tables in the Fabric data warehouse currently has some limitations. See [Tables in data warehousing in Microsoft Fabric](https://learn.microsoft.com/en-us/fabric/data-warehouse/tables) for more information.



# Query and transform data
There are two ways to query data from your data warehouse:

**1. The Visual query editor:** provides a **no-code**, drag-and-drop experience to create your queries. 

**2. SQL query editor:** to write **T-SQL** queries. 

In both cases, you can create tables, views, and stored procedures to query data in the data warehouse and Lakehouse.

There's also a **`SQL analytics endpoint`**, where you can connect from any tool.

## Visual query editor
**The Visual query editor:** provides an experience similar to the [Power Query online diagram view](https://learn.microsoft.com/en-us/power-query/diagram-view). 
- Use the **New visual query button** to create a new query.
- Drag a table from your data warehouse onto the canvas to get started. You can then use the **Transform** menu at the top of the screen to add columns, filters, and other transformations to your query. 
- You can use the (+) button on the visual itself to perform similar transformations.

![Visual Query Editor](./images/10/visual-query.png)

## Query data using the SQL query editor
**The SQL query editor:** provides a query experience that includes intellisense, code completion, syntax highlighting, client-side parsing, and validation. If you've written T-SQL in SQL Server Management Studio (SSMS) or Azure Data Studio (ADS), you'll find it familiar.
- To create a new query, use the **New SQL query button** in the menu. 
- You can author and run your T-SQL queries here. In the example below we're creating a new view for analysts to use for reporting in Power BI.

![SQL Query Editor](./images/10/create-view.png)

# Prepare data for analysis and reporting
You can easily switch between the **Data**, **Query**, and **Model** view Fabric using the menu in the bottom left corner of the screen. 
- The **Data** view shows the **`tables`** in the semantic model
- The **Query** view shows the **`SQL queries`** that are used to create the semantic model
- The **Model** view shows the **`semantic model`**.

## Semantic model
- Every time a **data warehouse** is **`created`**, Fabric **`creates`** a **semantic model** for analysts and/or business users to connect to for reporting.
- **Semantic models** are automatically kept in **`sync`** with the **data warehouse**, so you don't have to worry about maintaining them. You can also create **`custom` semantic models** to meet your specific needs.

### Understand semantic models
**A semantic model:** defines 
- The **`relationships`** between the different tables in the semantic model, 
- The **`rules`** for how data is aggregated and summarized, and 
- The **`calculations`** or **`measures`** that are used to derive insights from the data.

### Understand the default semantic model
There's also a **`default` semantic model** automatically created for you in Fabric. 
- It **inherits** business logic from the parent lakehouse or warehouse, which initiates the downstream analytics experience for business intelligence and analysis. This semantic model is **managed**, **optimized**, and **kept in sync** for you.
- New tables in the Lakehouse are **automatically added** to the default semantic model.
  - Users can also manually select tables or views from the warehouse they want included in the model for more flexibility.
  - Objects that are in the default semantic model are created as a layout in the model view.

<img src="https://files.training.databricks.com/images/icon_note_32.png" alt="Note"> Default semantic models follow the current limitations for semantic models in Power BI. See [Default Power BI semantic models](https://learn.microsoft.com/en-us/fabric/data-warehouse/limitations) for more information.

## Build relationships
**Relationships** allow you to **`connect tables`** in the semantic model. Create relationships between tables in your data warehouse using a click-and-drag interface in Fabric in the **Model** view.

![Fabric displaying relationships](./images/10/create-relationships.png)

<img src="https://files.training.databricks.com/images/icon_note_32.png" alt="Note"> See [Create and manage relationships](https://learn.microsoft.com/en-us/power-bi/transform-model/desktop-create-and-manage-relationships) for detailed information on creating relationships.

## Create measures
**Measures** are the **metrics** that you want to **`analyze`** in your data warehouse. You can create measures in Fabric by using the New measure button in the **Model** view.
- Measures are calculated fields that are based on the data in the tables in your data warehouse using the **Data Analysis Expressions (DAX)** formula language.

![Fabric displaying the new measure](./images/10/create-measure.png)

Fabric offers many tools to create data transformations. The creation of measures using DAX is one of many ways to create data transformations. To learn more about DAX, see [Use DAX in Power BI](https://learn.microsoft.com/en-us/training/paths/dax-power-bi/).

## Hide fields
Building out the semantic model is a critical component to preparing your data for use in downstream reporting. To simplify things for your report builders, you can **`hide elements`** from **view**, either a **table** or a **column**. Right-click on the table or column and select **Hide**. 
- Hiding fields removes the table or column from the **model view**, but it will still be available for use in the semantic model.

![Hidden fields](./images/10/hide-fields.png)

## Visualize data
Fabric enables you to **`visualize`** the results of a **single query** or your **entire data warehouse**, without leaving the data warehouse experience.

Use the **New report** button to create a new Power BI report from the contents of your entire data warehouse. Using the **New report** button opens the Power BI service experience where you can build and save your report for use by the business.

![Power BI report](./images/10/sales-report.png)

# Secure and monitor your data warehouse

## Security
**Data warehouse security:** is important to protect your data from unauthorized access. Fabric provides a number of security features to help you secure your data warehouse. These include:

- **Role-based access control (RBAC):** to **control `access`** to the warehouse and its data.
- **SSL encryption:** to **secure** the **`communication`** between the **warehouse** and the **client applications**.
- **Azure Storage Service Encryption:** to **protect** the **`data`** in transit and at rest.
- **Azure Monitor and Azure Log Analytics:** to **monitor** the warehouse **`activity`** and **`audit`** the access to the data.
- **Multifactor authentication (MFA):** to **add** an **`extra layer`** of security to user accounts.
- **Microsoft Entra ID integration:** to **manage** the user **`identities`** and **`access`** to the warehouse.

### Workspace permissions
Data in Fabric is organized into **workspaces**, which are used to 
- **control `access`** and 
- **manage** the **`lifecycle of data`** and **`services`**. 

Appropriate **`workspace roles`** are the **`first` line of defense** in securing your data warehouse.

<img src="https://files.training.databricks.com/images/icon_note_32.png" alt="Note"> See [Workspaces in Power BI](https://learn.microsoft.com/en-us/power-bi/collaborate-share/service-new-workspaces#roles-and-licenses) for more information on workspace roles.

### Item permissions
In contrast to **`workspace` roles**, which apply to **`all` items** within a workspace, you can use **`item` permissions** to grant access to **`individual` warehouses**. 
- This enables you to share a single data warehouse for downstream consumption.
- You can grant permissions to users via T-SQL or in the Fabric portal. 

Grant the following permissions to users who need to access your data warehouse:

- **Read:** Allows the user to **connect** using the SQL connection string.
- **ReadData:** Allows the user to **read** data from any **`table/view`** within the warehouse.
- **ReadAll:** Allows user to **read** data the **`raw parquet files`** in OneLake that can be consumed by Spark.

A user connection to the **`SQL analytics endpoint`** will **fail without `Read` permission** at a minimum.

## Monitoring
**Monitoring activities** in your data warehouse is crucial to:
- Ensure optimal performance, efficient resource utilization, and security.
- Identify issues, detect anomalies, and take action to keep the data warehouse running smoothly and securely.

You can use **`dynamic management views (DMVs)`** to monitor connection, session, and request status to see live SQL query lifecycle insights. There are currently three DMVs available to use in Fabric:

- **`sys.dm_exec_connections`:** Returns information about each **`connection`** established between the warehouse and the engine.
- **`sys.dm_exec_sessions`:** Returns information about each **`session`** authenticated between the item and engine.
- **`sys.dm_exec_requests`:** Returns information about each active **`request`** in a session.

### Query monitoring
Use 'sys.dm_exec_requests' to identify long-running queries that may be impacting the overall performance of the database, and take appropriate action to optimize or terminate those queries.

In [0]:
%sql
-- Start by identifying the queries that have been running for a long time. Use the following query to identify which queries have been running the longest, in descending order:
SELECT request_id, session_id, start_time, total_elapsed_time
    FROM sys.dm_exec_requests
    WHERE status = 'running'
    ORDER BY total_elapsed_time DESC;

In [0]:
%sql
-- You can continue investigating to understand which user ran the session with the long-running query, by running:
SELECT login_name
    FROM sys.dm_exec_sessions
    WHERE 'session_id' = 'SESSION_ID WITH LONG-RUNNING QUERY';

In [0]:
%sql
-- Finally, you can use the KILL command to terminate the session with the long-running query:
KILL 'SESSION_ID WITH LONG-RUNNING QUERY';

<img src="https://files.training.databricks.com/images/icon_note_32.png" alt="Note"> You must be a workspace Admin to run the `KILL` command. Workspace Admins can execute all three DMVs. Member, Contributor, and Viewer roles can see their own results within the warehouse, but cannot see other users' results.